In [1]:
from iism_lab2 import a5_generator
import numpy as np


def get_a5_generator():
    S19 = np.random.randint(2, size=19)
    S22 = np.random.randint(2, size=22)
    S23 = np.random.randint(2, size=23)
    gen = a5_generator(S19, S22, S23)
    return gen


def uniform_psi(gen, n=10):
    while True:
        yield (int(''.join([str(next(gen)) for _ in range(n)]), 2) + 1) / (2 ** n + 1)

$$\int_{88}^{99} \ln{(x)}\cdot\sin{(x)} dx = \int_{88}^{99} \frac{1}{11} \cdot 11 \cdot \ln{(x)}\cdot \sin{(x)} = E\{11\cdot\ln{(x)}\cdot\sin{(x)}\}\approx \frac{1}{n}\sum_{i=1}^{n} 11 \cdot \ln{(x_i)}\sin{(x_i)}$$

In [15]:
psi = uniform_psi(get_a5_generator())
exps = np.array([11 * next(psi) + 88 for _ in range(2000)])
print(np.mean(exps))

93.521962439


$4.2818$

In [16]:
value = np.mean(11 * np.log(exps) * np.sin(exps))
print(value)

4.3075411555


In [29]:
exps = np.random.uniform(low=88, high=99, size=100000)
value = np.mean(11 * np.log(exps) * np.sin(exps))
print(value)

4.26459149058


$$\iint\limits_{|x| + |y| < 1}\frac{x^3+3xy}{e^{-y}}dxdy = \iint\limits_{|x| + |y| < 1} \frac{1}{2} \cdot 2 \cdot \frac{x^3 + 3xy}{e^{-y}}dxdy \approx \frac{1}{n} \sum\limits_{i=1}^{n} 2 \cdot \frac{x_i^3 + 3x_iy_i}{e^{-y_i}}$$

Правильное значение: $0$

In [4]:
exps = np.random.uniform(low=-1/2**0.5, high=1/2**0.5, size=(10000, 2))
M = np.array([[np.cos(np.pi / 4), -np.sin(np.pi / 4)], 
              [np.sin(np.pi / 4), np.cos(np.pi / 4)]])
for i in range(len(exps)):
    exps[i] = (M.dot(exps[i].reshape((2, 1)))).reshape((1, 2))
x, y = exps.reshape((2, 10000))
value = np.mean(2 * (x ** 3 + 3 * x * y) / np.exp(-y))
print(value)

1.20185757916e-05
